<a href="https://colab.research.google.com/github/Tien84/Face-Detection/blob/master/GridSearch_SARIMA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas

In [ ]:
!pip install numpy

In [ ]:
!pip install matplotlib

In [ ]:
!pip install sklearn

In [1]:
from google.colab import drive
drive.mount('/content/driver')

Mounted at /content/driver


In [2]:
# grid search sarima hyperparameters for daily female dataset
from math import sqrt
from multiprocessing import cpu_count
from joblib import Parallel
from joblib import delayed
from warnings import catch_warnings
from warnings import filterwarnings
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from pandas import read_csv

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [3]:
# one-step sarima forecast
def sarima_forecast(history, config):
	order, sorder, trend = config
	# define model
	model = SARIMAX(history, order=order, seasonal_order=sorder, trend=trend, enforce_stationarity=False, enforce_invertibility=False)
	# fit model
	model_fit = model.fit(disp=False)
	# make one step forecast
	yhat = model_fit.predict(len(history), len(history))
	return yhat[0]

In [4]:
# root mean squared error or rmse
def measure_rmse(actual, predicted):
	return sqrt(mean_squared_error(actual, predicted))

In [5]:
# split a univariate dataset into train/test sets
def train_test_split(data, n_test):
	return data[:-n_test], data[-n_test:]

In [6]:
# walk-forward validation for univariate data
def walk_forward_validation(data, n_test, cfg):
	predictions = list()
	# split dataset
	train, test = train_test_split(data, n_test)
	# seed history with training dataset
	history = [x for x in train]
	# step over each time-step in the test set
	for i in range(len(test)):
		# fit model and make forecast for history
		yhat = sarima_forecast(history, cfg)
		# store forecast in list of predictions
		predictions.append(yhat)
		# add actual observation to history for the next loop
		history.append(test[i])
	# estimate prediction error
	error = measure_rmse(test, predictions)
	return error

In [7]:
# score a model, return None on failure
def score_model(data, n_test, cfg, debug=False):
	result = None
	# convert config to a key
	key = str(cfg)
	# show all warnings and fail on exception if debugging
	if debug:
		result = walk_forward_validation(data, n_test, cfg)
	else:
		# one failure during model validation suggests an unstable config
		try:
			# never show warnings when grid searching, too noisy
			with catch_warnings():
				filterwarnings("ignore")
				result = walk_forward_validation(data, n_test, cfg)
		except:
			error = None
	# check for an interesting result
	if result is not None:
		print(' > Model[%s] %.3f' % (key, result))
	return (key, result)

In [8]:
# grid search configs
def grid_search(data, cfg_list, n_test, parallel=True):
	scores = None
	if parallel:
		# execute configs in parallel
		executor = Parallel(n_jobs=cpu_count(), backend='multiprocessing')
		tasks = (delayed(score_model)(data, n_test, cfg) for cfg in cfg_list)
		scores = executor(tasks)
	else:
		scores = [score_model(data, n_test, cfg) for cfg in cfg_list]
	# remove empty results
	scores = [r for r in scores if r[1] != None]
	# sort configs by error, asc
	scores.sort(key=lambda tup: tup[1])
	return scores

In [9]:
# create a set of sarima configs to try
def sarima_configs(seasonal=[0]):
	models = list()
	# define config lists
	p_params = [0, 1, 2]
	d_params = [0, 1]
	q_params = [0, 1, 2]
	t_params = ['n','c','t','ct']
	P_params = [0, 1, 2]
	D_params = [0, 1]
	Q_params = [0, 1, 2]
	m_params = seasonal
	# create config instances
	for p in p_params:
		for d in d_params:
			for q in q_params:
				for t in t_params:
					for P in P_params:
						for D in D_params:
							for Q in Q_params:
								for m in m_params:
									cfg = [(p,d,q), (P,D,Q,m), t]
									models.append(cfg)
	return models

In [ ]:
if __name__ == '__main__':
	# load dataset
	series = read_csv('/content/driver/My Drive/LSTM_Model/AAPL.csv', header=0, index_col=0)
	data = series['Adj Close'].values
	print(data.shape)
	# data split
	#n_test = 165
	n_test = 673
	# model configs
	cfg_list = sarima_configs()
	# grid search
	scores = grid_search(data, cfg_list, n_test)
	print('done')
	# list top 3 configs
	for cfg, error in scores[:10]:
		print(cfg, error)

(2018,)
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'n']] 189.480
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'n']] 95.770
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'n']] 2.982
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'n']] 2.986
 > Model[[(0, 0, 0), (0, 0, 2, 0), 'n']] 51.814
 > Model[[(0, 0, 0), (1, 0, 2, 0), 'n']] 2.984
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'n']] 2.986
 > Model[[(0, 0, 0), (2, 0, 2, 0), 'n']] 2.985
 > Model[[(0, 0, 0), (0, 0, 0, 0), 'c']] 83.382
 > Model[[(0, 0, 0), (2, 0, 1, 0), 'n']] 2.986
 > Model[[(0, 0, 0), (0, 0, 1, 0), 'c']] 42.758
 > Model[[(0, 0, 0), (0, 0, 2, 0), 'c']] 25.179
 > Model[[(0, 0, 0), (1, 0, 0, 0), 'c']] 2.985
 > Model[[(0, 0, 0), (1, 0, 1, 0), 'c']] 3.141
 > Model[[(0, 0, 0), (1, 0, 2, 0), 'c']] 5.754
 > Model[[(0, 0, 0), (2, 0, 0, 0), 'c']] 2.994
